# Imports

In [1]:
config = {
    'dhdt': {
        'depth': 3,
        'learning_rate': 0.01,#1e-3,
        
        'initializer': 'he_normal', #GlorotUniform
        
        'loss': 'binary_crossentropy',#'mae',
        'optimizer': 'adam',        
        
        'beta_1': 10,
        'beta_2': 50,
        
        'squeeze_factor': 1,
        
        'batch_size': 512,
        'epochs': 1_000,
        'early_stopping_epochs': 50,
    },
    
    
    
    'make_classification': {
        'number_of_variables': 10,
        'n_samples': 5_000,
        'num_eval': 50,
    },

    'computation': {
        'random_seed': 42,
        'trials': 5,
        'n_jobs': 50,
        'verbosity': 0,
    },
}



In [2]:
import numpy as np

import sklearn
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder, OrdinalEncoder

from livelossplot import PlotLosses

import os
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

from IPython.display import Image
from IPython.display import display, clear_output

import pandas as pd

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = '' #'true'

import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import logging

import tensorflow as tf
import tensorflow_addons as tfa

tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(3)

np.seterr(all="ignore")

from keras import backend as K
from keras.utils.generic_utils import get_custom_objects


import seaborn as sns
sns.set_style("darkgrid")

import time
import random

from utilities.utilities import *
from utilities.DHDT import *

from joblib import Parallel, delayed

from itertools import product
from collections.abc import Iterable

from copy import deepcopy

# Evaluation

## make_classification

In [3]:
if True:
    metrics = ['accuracy', 'f1']
    
    config_test = deepcopy(config)
    config_test['make_classification']['n_samples'] = 10_000
    config_test['dhdt']['epochs'] = 100

    dataset_dict = {}
    model_dict = {}

    scores_dict = {'sklearn': {},
                   'DHDT': {}}

    dataset_dict = get_preprocessed_dataset('make_classification',
                                            random_seed=config_test['computation']['random_seed'],
                                            config=config_test['make_classification'],
                                            verbosity=1)

    model_dict['sklearn'] = DecisionTreeClassifier(max_depth=3, 
                                                   random_state=config_test['computation']['random_seed'])

    model_dict['sklearn'].fit(dataset_dict['X_train'], 
                              dataset_dict['y_train'])



    model_dict['DHDT'] = DHDT(dataset_dict['X_train'].shape[1],

                                depth = config_test['dhdt']['depth'],

                                learning_rate = config_test['dhdt']['learning_rate'],
                                optimizer = config_test['dhdt']['optimizer'],

                                beta_1 = config_test['dhdt']['beta_1'],
                                beta_2 = config_test['dhdt']['beta_2'],

                                squeeze_factor = config_test['dhdt']['squeeze_factor'],

                                loss = config_test['dhdt']['loss'],#'mae',

                                random_seed = config_test['computation']['random_seed'],
                                verbosity = 2)        


    scores_dict['history'] = model_dict['DHDT'].fit(dataset_dict['X_train'], 
                                                  dataset_dict['y_train'], 
                                                  batch_size=config_test['dhdt']['batch_size'], 
                                                  epochs=config_test['dhdt']['epochs'], 
                                                  early_stopping_epochs=config_test['dhdt']['early_stopping_epochs'], 
                                                  valid_data=(dataset_dict['X_valid'], dataset_dict['y_valid']))



    dataset_dict['y_test_dhdt'] = model_dict['DHDT'].predict(dataset_dict['X_test'])
    dataset_dict['y_valid_dhdt'] = model_dict['DHDT'].predict(dataset_dict['X_valid'])

    dataset_dict['y_test_sklearn'] = model_dict['sklearn'].predict(dataset_dict['X_test'])
    dataset_dict['y_valid_sklearn'] = model_dict['sklearn'].predict(dataset_dict['X_valid'])     
    
    for metric in metrics:
        
        if metric in ['accuracy', 'f1']:
            y_test_dhdt = np.round(dataset_dict['y_test_dhdt'])
            y_valid_dhdt = np.round(dataset_dict['y_valid_dhdt'])
            y_test_sklearn = np.round(dataset_dict['y_test_sklearn'])
            y_valid_sklearn = np.round(dataset_dict['y_valid_sklearn'])         
        else:
            y_test_dhdt = dataset_dict['y_test_dhdt']
            y_valid_dhdt = dataset_dict['y_valid_dhdt']
            y_test_sklearn = dataset_dict['y_test_sklearn']
            y_valid_sklearn =    dataset_dict['y_valid_sklearn']                
        
        scores_dict['sklearn'][metric + '_test'] = sklearn.metrics.get_scorer(metric)._score_func(dataset_dict['y_test'], y_test_sklearn)
        scores_dict['DHDT'][metric + '_test'] = sklearn.metrics.get_scorer(metric)._score_func(dataset_dict['y_test'], y_test_dhdt)

        scores_dict['sklearn'][metric + '_valid'] = sklearn.metrics.get_scorer(metric)._score_func(dataset_dict['y_valid'], y_valid_sklearn)   
        scores_dict['DHDT'][metric + '_valid'] = sklearn.metrics.get_scorer(metric)._score_func(dataset_dict['y_valid'], y_valid_dhdt)


Original Data Shape (selected):  (10000, 10)
Original Data Shape (encoded):  (10000, 10)
Original Data Class Distribution:  4993  (true) / 5007  (false)
(7000, 10) (7000,)
(1000, 10) (1000,)
(2000, 10) (2000,)
True Ratio:  0.501


epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 00 | Loss: 0.69344 |
Epoch: 01 | Loss: 0.69307 |
Epoch: 02 | Loss: 0.69250 |
Epoch: 03 | Loss: 0.69021 |
Epoch: 04 | Loss: 0.68434 |
Epoch: 05 | Loss: 0.67505 |
Epoch: 06 | Loss: 0.66785 |
Epoch: 07 | Loss: 0.66171 |
Epoch: 08 | Loss: 0.65201 |
Epoch: 09 | Loss: 0.63913 |
Epoch: 10 | Loss: 0.62264 |
Epoch: 11 | Loss: 0.61084 |
Epoch: 12 | Loss: 0.60216 |
Epoch: 13 | Loss: 0.59694 |
Epoch: 14 | Loss: 0.59186 |
Epoch: 15 | Loss: 0.59031 |
Epoch: 16 | Loss: 0.58694 |
Epoch: 17 | Loss: 0.58578 |
Epoch: 18 | Loss: 0.58523 |
Epoch: 19 | Loss: 0.58461 |
Epoch: 20 | Loss: 0.58262 |
Epoch: 21 | Loss: 0.58175 |
Epoch: 22 | Loss: 0.58159 |
Epoch: 23 | Loss: 0.58235 |
Epoch: 24 | Loss: 0.58160 |
Epoch: 25 | Loss: 0.58071 |
Epoch: 26 | Loss: 0.58069 |
Epoch: 27 | Loss: 0.58045 |
Epoch: 28 | Loss: 0.58123 |
Epoch: 29 | Loss: 0.58123 |
Epoch: 30 | Loss: 0.57994 |
Epoch: 31 | Loss: 0.58070 |
Epoch: 32 | Loss: 0.58129 |
Epoch: 33 | Loss: 0.58151 |
Epoch: 34 | Loss: 0.58136 |
Epoch: 35 | Loss: 0.

In [4]:
if True:
    parallel_eval_synthetic = Parallel(n_jobs=config['computation']['n_jobs'], verbose=3, backend='loky') #loky #sequential multiprocessing
    evaluation_results_synthetic = parallel_eval_synthetic(delayed(evaluate_synthetic_parallel)(index = index,
                                                                                                random_seed_data = config['computation']['random_seed']+index,
                                                                                                random_seed_model = config['computation']['random_seed'],#+random_seed_model,
                                                                                                config = config,
                                                                                                verbosity = -1) for index in range(config['make_classification']['num_eval']))

    for i, synthetic_result in enumerate(evaluation_results_synthetic):
        if i == 0:
            model_dict_synthetic = synthetic_result[0]
            scores_dict_synthetic = synthetic_result[1]
            dataset_dict_synthetic = synthetic_result[2]
        else: 
            model_dict_synthetic = mergeDict(model_dict_synthetic, synthetic_result[0])
            scores_dict_synthetic = mergeDict(scores_dict_synthetic, synthetic_result[1])
            dataset_dict_synthetic = mergeDict(dataset_dict_synthetic, synthetic_result[2])    
    
    del synthetic_result, evaluation_results_synthetic
    
    
    metric_identifer = '_test'
    metrics = ['accuracy', 'f1']
    index = [i for i in range(config['make_classification']['num_eval'])]
    columns = flatten_list([[[approach + ' ' + metric + '_mean', approach + ' ' + metric + '_max', approach + ' ' + metric + '_std'] for metric in metrics] for approach in ['DHDT', 'sklearn']])


    results_DHDT = None
    results_sklearn = None
    for metric in metrics:
        scores_DHDT = [scores_dict_synthetic[i]['DHDT'][metric + metric_identifer] for i in range(config['make_classification']['num_eval'])]

        scores_sklearn = [scores_dict_synthetic[i]['sklearn'][metric + metric_identifer] for i in range(config['make_classification']['num_eval'])]

        scores_DHDT_mean = np.mean(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
        scores_sklearn_mean = np.mean(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

        scores_DHDT_max = np.max(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
        scores_sklearn_max = np.max(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

        scores_DHDT_std = np.std(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['computation']['trials'])
        scores_sklearn_std = np.std(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['computation']['trials'])

        results_DHDT_by_metric = np.vstack([scores_DHDT_mean, scores_DHDT_max, scores_DHDT_std])
        results_sklearn_by_metric = np.vstack([scores_sklearn_mean, scores_sklearn_max, scores_sklearn_std])

        if results_DHDT is None and results_sklearn is None:
            results_DHDT = results_DHDT_by_metric
            results_sklearn = results_sklearn_by_metric
        else:
            results_DHDT = np.vstack([results_DHDT, results_DHDT_by_metric])
            results_sklearn = np.vstack([results_sklearn, results_sklearn_by_metric])

    scores_dataframe_synthetic = pd.DataFrame(data=np.vstack([results_DHDT, results_sklearn]).T, index = index, columns = columns)    
    
    del model_dict_synthetic, scores_dict_synthetic, dataset_dict_synthetic
    
    display(scores_dataframe_synthetic)
    display(scores_dataframe_synthetic[scores_dataframe_synthetic.columns[1::3]])
    display(scores_dataframe_synthetic.describe())

[Parallel(n_jobs=50)]: Using backend LokyBackend with 50 concurrent workers.
[Parallel(n_jobs=50)]: Done   2 out of  50 | elapsed:  4.6min remaining: 111.3min
[Parallel(n_jobs=50)]: Done  19 out of  50 | elapsed:  5.3min remaining:  8.6min
[Parallel(n_jobs=50)]: Done  36 out of  50 | elapsed:  5.8min remaining:  2.2min
[Parallel(n_jobs=50)]: Done  50 out of  50 | elapsed:  6.7min finished


,DHDT accuracy_mean,DHDT accuracy_max,DHDT accuracy_std,DHDT f1_mean,DHDT f1_max,DHDT f1_std,sklearn accuracy_mean,sklearn accuracy_max,sklearn accuracy_std,sklearn f1_mean,sklearn f1_max,sklearn f1_std
0,0.6976,0.726,0.030689,0.683065,0.720077,0.043618,0.7782,0.843,0.0324,0.758970,0.845624,0.043327
1,0.7418,0.817,0.079300,0.722924,0.826211,0.109143,0.8354,0.849,0.0272,0.833075,0.856600,0.047050
2,0.7178,0.749,0.026195,0.709361,0.736067,0.032590,0.8030,0.815,0.0060,0.795694,0.818805,0.011556
3,0.7476,0.786,0.021191,0.738520,0.773784,0.024413,0.8078,0.855,0.0236,0.790421,0.862819,0.036199
4,0.5994,0.707,0.056191,0.551282,0.697626,0.143660,0.6818,0.809,0.0636,0.699457,0.828083,0.064313
5,0.7664,0.802,0.047861,0.754428,0.803828,0.071263,0.8292,0.840,0.0216,0.834094,0.844358,0.020528
6,0.7106,0.764,0.033073,0.734459,0.752036,0.012276,0.7714,0.817,0.0228,0.799261,0.816817,0.008778
7,0.7586,0.801,0.039510,0.743214,0.805854,0.037219,0.8472,0.848,0.0016,0.836330,0.836820,0.000245
8,0.8348,0.868,0.060970,0.827023,0.865580,0.066096,0.8484,0.867,0.0372,0.855181,0.876279,0.042195
9,0.6924,0.749,0.032364,0.644781,0.766076,0.068281,0.7958,0.831,0.0176,0.799897,0.838894,0.019499


,DHDT accuracy_max,DHDT f1_max,sklearn accuracy_max,sklearn f1_max
0,0.726,0.720077,0.843,0.845624
1,0.817,0.826211,0.849,0.856600
2,0.749,0.736067,0.815,0.818805
3,0.786,0.773784,0.855,0.862819
4,0.707,0.697626,0.809,0.828083
5,0.802,0.803828,0.840,0.844358
6,0.764,0.752036,0.817,0.816817
7,0.801,0.805854,0.848,0.836820
8,0.868,0.865580,0.867,0.876279
9,0.749,0.766076,0.831,0.838894


,DHDT accuracy_mean,DHDT accuracy_max,DHDT accuracy_std,DHDT f1_mean,DHDT f1_max,DHDT f1_std,sklearn accuracy_mean,sklearn accuracy_max,sklearn accuracy_std,sklearn f1_mean,sklearn f1_max,sklearn f1_std
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.685896,0.748940,0.049860,0.660412,0.747923,0.084916,0.774644,0.806340,0.024224,0.774672,0.809056,0.026574
std,0.055002,0.056871,0.021304,0.073283,0.054148,0.054057,0.052176,0.047979,0.017181,0.054718,0.045482,0.019852
min,0.583800,0.616000,0.021160,0.490955,0.629396,0.011228,0.660400,0.670000,0.000800,0.589275,0.685714,0.000245
25%,0.649600,0.709250,0.032624,0.616079,0.715651,0.045723,0.740300,0.781250,0.009200,0.744420,0.776096,0.010459
50%,0.689700,0.752500,0.046316,0.659998,0.746514,0.071745,0.781800,0.815500,0.021200,0.787450,0.818190,0.021025
75%,0.719350,0.784750,0.061731,0.719380,0.781096,0.105724,0.815750,0.839750,0.035300,0.815679,0.840912,0.041889
max,0.834800,0.868000,0.103779,0.827023,0.865616,0.288036,0.856800,0.890000,0.063600,0.855181,0.885655,0.084483


## Real-World Eval

In [5]:
if True:

    identifier_list = [
                        'Adult',#: 32,
                        'Bank Marketing',#: 32,
                        'Loan Credit',#: 32,

                        'Credit Card',#: 23, 
                        'Car',#: 21,


                        'Absenteeism',#: 15,
                        'Loan House',#: 15,
                        'Cervical Cancer',#: 15,

                        'Heart Disease',#: 13,           

                        'Titanic',#: 10,
                        'Medical Insurance',#: 10,
                        'Wisconsin Breast Cancer Original',#: 10,
                        'Wisconsin Diagnostic Breast Cancer',#: 10,
                        'Wisconsin Prognostic Breast Cancer',#: 10,
                        'Abalone',#: 10,

                        'Habermans Survival',#: 3, 
                      ]

    identifier_list = ['Habermans Survival']

    parallel_eval_real_world = Parallel(n_jobs=config['computation']['n_jobs'], verbose=3, backend='loky') #loky #sequential multiprocessing
    evaluation_results_real_world = parallel_eval_real_world(delayed(evaluate_real_world_parallel)(identifier_list=identifier_list, 
                                                                                                   random_seed_model=config['computation']['random_seed']+i,
                                                                                                   config = config,
                                                                                                   verbosity = -1) for i in range(config['computation']['trials']))


    for i, real_world_result in enumerate(evaluation_results_real_world):
        if i == 0:
            model_dict_real_world = real_world_result[0]
            scores_dict_real_world = real_world_result[1]
            dataset_dict_real_world = real_world_result[2]
        else: 
            model_dict_real_world = mergeDict(model_dict_real_world, real_world_result[0])
            scores_dict_real_world = mergeDict(scores_dict_real_world, real_world_result[1])
            dataset_dict_real_world = mergeDict(dataset_dict_real_world, real_world_result[2])    

    del real_world_result, evaluation_results_real_world

    metric_identifer = '_test'
    metrics = ['accuracy', 'f1']
    index = identifier_list
    columns = flatten_list([[[approach + ' ' + metric + '_mean', approach + ' ' + metric + '_max', approach + ' ' + metric + '_std'] for metric in metrics] for approach in ['DHDT', 'sklearn']])


    results_DHDT = None
    results_sklearn = None
    for metric in metrics:
        scores_DHDT = [scores_dict_real_world[identifier]['DHDT'][metric + metric_identifer] for identifier in identifier_list]

        scores_sklearn = [scores_dict_real_world[identifier]['sklearn'][metric + metric_identifer] for identifier in identifier_list]    

        scores_DHDT_mean = np.mean(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
        scores_sklearn_mean = np.mean(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

        scores_DHDT_max = np.max(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
        scores_sklearn_max = np.max(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

        scores_DHDT_std = np.std(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['computation']['trials'])
        scores_sklearn_std = np.std(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['computation']['trials'])

        results_DHDT_by_metric = np.vstack([scores_DHDT_mean, scores_DHDT_max, scores_DHDT_std])
        results_sklearn_by_metric = np.vstack([scores_sklearn_mean, scores_sklearn_max, scores_sklearn_std])

        if results_DHDT is None and results_sklearn is None:
            results_DHDT = results_DHDT_by_metric
            results_sklearn = results_sklearn_by_metric
        else:
            results_DHDT = np.vstack([results_DHDT, results_DHDT_by_metric])
            results_sklearn = np.vstack([results_sklearn, results_sklearn_by_metric])

    del model_dict_real_world, scores_dict_real_world, dataset_dict_real_world
            
    scores_dataframe_real_world = pd.DataFrame(data=np.vstack([results_DHDT, results_sklearn]).T, index = index, columns = columns)
    display(scores_dataframe_real_world)
    display(scores_dataframe_real_world[scores_dataframe_real_world.columns[1::3]])    




[Parallel(n_jobs=50)]: Using backend LokyBackend with 50 concurrent workers.
[Parallel(n_jobs=50)]: Done   2 out of   5 | elapsed:   26.7s remaining:   40.1s
[Parallel(n_jobs=50)]: Done   5 out of   5 | elapsed:   27.3s finished


,DHDT accuracy_mean,DHDT accuracy_max,DHDT accuracy_std,DHDT f1_mean,DHDT f1_max,DHDT f1_std,sklearn accuracy_mean,sklearn accuracy_max,sklearn accuracy_std,sklearn f1_mean,sklearn f1_max,sklearn f1_std
Habermans Survival,0.695082,0.704918,0.008031,0.82009,0.826923,0.005579,0.688525,0.688525,0.0,0.791209,0.791209,0.0


,DHDT accuracy_max,DHDT f1_max,sklearn accuracy_max,sklearn f1_max
Habermans Survival,0.704918,0.826923,0.688525,0.791209


In [6]:
if False:
    identifier = identifier_list[0]#"Absenteeism"
    plt.figure(figsize=(15,8))
    image = model_dict_real_world[identifier]['DHDT'].plot(normalizer_list=dataset_dict_real_world[identifier]['normalizer_list'])
    display(image)

    plt.figure(figsize=(15,8))
    plot_tree(model_dict_real_world[identifier]['sklearn'], fontsize=10) 
    plt.show()